In [ ]:
import os
import shutil
import argparse
import logging

import torch
import torch.utils.data as data
import numpy as np

from utils.settings import Settings


settings_file_name = 'settings.ini'
settings = Settings(settings_file_name)
data_parameters = settings['DATA']
training_parameters = settings['TRAINING']
network_parameters = settings['NETWORK']
misc_parameters = settings['MISC']

In [ ]:
import os
import shutil
import argparse
import logging

import torch
import torch.utils.data as data
import numpy as np

from utils.settings import Settings


settings_file_name = 'settings.ini'
settings = Settings(settings_file_name)
data_parameters = settings['DATA']
training_parameters = settings['TRAINING']
network_parameters = settings['NETWORK']
misc_parameters = settings['MISC']

number_of_modalities = 2
batch_size = 1
network_parameters['network_number'] = 1

if data_parameters['fused_data_flag'] == True or network_parameters['network_number']<=2:
    original_input_channels = number_of_modalities
    X = torch.rand(batch_size, number_of_modalities, 160, 192, 160)
else:
    original_input_channels = number_of_modalities
    X = {}
    for idx in range(number_of_modalities):
        X[idx] = torch.rand(batch_size, 1, 160, 192, 160)
    
if network_parameters['network_number'] == 1:
    from MultiAgeMapper import AgeMapper_input1
    AgeMapperModel = AgeMapper_input1(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = original_input_channels
                                )
    print('--> Using NETWORK 1')
elif network_parameters['network_number'] == 2:
    from MultiAgeMapper import AgeMapper_input2
    AgeMapperModel = AgeMapper_input2(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = original_input_channels,
                                network_2_modality_filter_outputs = network_parameters['network_2_modality_filter_outputs']
                                )
    print('--> Using NETWORK 2')
elif network_parameters['network_number'] == 3:
    from MultiAgeMapper import AgeMapper_input3
    AgeMapperModel = AgeMapper_input3(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities
                                )
    print('--> Using NETWORK 3')
elif network_parameters['network_number'] == 4:
    from MultiAgeMapper import AgeMapper_input4
    AgeMapperModel = AgeMapper_input4(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities
                                )
    print('--> Using NETWORK 4')
elif network_parameters['network_number'] == 5:
    from MultiAgeMapper import AgeMapper_input5
    AgeMapperModel = AgeMapper_input5(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities
                                )
    print('--> Using NETWORK 5')
elif network_parameters['network_number'] == 6:
    from MultiAgeMapper import AgeMapper_input6
    AgeMapperModel = AgeMapper_input6(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities
                                )
    print('--> Using NETWORK 6')
elif network_parameters['network_number'] == 7:
    from MultiAgeMapper import AgeMapper_input7
    AgeMapperModel = AgeMapper_input7(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities
                                )
    print('--> Using NETWORK 7')
elif network_parameters['network_number'] == 8:
    from MultiAgeMapper import AgeMapper_input8
    AgeMapperModel = AgeMapper_input8(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities
                                )
    print('--> Using NETWORK 8')
elif network_parameters['network_number'] == 9:
    from MultiAgeMapper import AgeMapper_input9
    AgeMapperModel = AgeMapper_input9(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities
                                )
    print('--> Using NETWORK 9')
    
print(AgeMapperModel)

AgeMapperModel.eval()
# with torch.no_grad():
#     y = AgeMapperModel(X)
#     print('y shape: ', y.shape)

model_parameters = filter(lambda p: p.requires_grad, AgeMapperModel.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

In [ ]:
# ORIGINAL: 1313345 / 2626690 (x2)

# Net1 - 1314209
# Net2 - 1313347
# Net3 - 2623457
# Net4 - 2626625
# Net5 - 2626689
# Net6 - 2626693
# Net7 - 186031457
# Net8 - 2642081
# Net9 - 2628737

In [ ]:
class AgeMapperConv(nn.Module):
    def __init__(
        self,
        channel_number=[32,64,64,64,64],
        path_number = 1
                ):

        super(AgeMapperConv, self).__init__()
        number_of_layers = len(channel_number)

        self.Feature_Extractor = nn.Sequential()
        for layer_number in range(number_of_layers):      
            if layer_number == 0:
                input_channels = 1
            else:
                input_channels = channel_number[layer_number - 1]
            output_channels = channel_number[layer_number]

            self.Feature_Extractor.add_module(
                name = 'Path_%d_Convolution_%d' % (path_number, layer_number),
                module = self._convolutional_block(
                    input_channels,
                    output_channels,
                    maxpool_flag = True,
                    kernel_size = 3,
                    padding_flag= True
                )
            )

    @staticmethod
    def _convolutional_block(input_channels, output_channels, maxpool_flag=True, kernel_size=3, padding_flag=True, maxpool_stride=2):
        
        if padding_flag == True:
            padding = int((kernel_size - 1) / 2)
        else:
            padding = 0
        
        if maxpool_flag is True:
            layer = nn.Sequential(
                nn.Conv3d(
                    in_channels=input_channels,
                    out_channels=output_channels,
                    kernel_size=kernel_size,
                    padding=padding,
                    bias=False
                ),
                nn.BatchNorm3d(
                    num_features=output_channels,
                    affine=True
                ),
                nn.MaxPool3d(
                    kernel_size=2,
                    stride=maxpool_stride
                ),
                nn.ReLU()
            )
        else:
            layer = nn.Sequential(
                nn.Conv3d(
                    in_channels=input_channels,
                    out_channels=output_channels,
                    kernel_size=kernel_size,
                    padding=padding,
                    bias=False
                ),
                nn.BatchNorm3d(
                    num_features=output_channels,
                    affine=True
                ),
                nn.ReLU()
            )

        return layer
    
    def forward(self, X):
        X = self.Feature_Extractor(X)
        return X


In [ ]:
fused_data_flag = False
original_input_channels = 2
channel_number=[32,64,64,64,64]
input_paths = original_input_channels

In [ ]:
Convolution_Paths = nn.ModuleDict({})
for idx in range(input_paths):
    Convolution_Paths[str(idx)] = AgeMapperConv(channel_number=channel_number,
                                                        path_number=idx)

In [ ]:
for idx in range(input_paths):
    print(idx)
    X[idx] = Convolution_Paths[str(idx)](X[idx])
    print(idx + 10000)
    if idx==0:
        X_output = X[idx].reshape(-1, prod(tensor(X.shape)[1:]))
    else:
        X_output = cat((X_output, X[idx].reshape(-1, prod(tensor(X.shape)[1:])) ), dim=1)

In [ ]:
import os
import shutil
import argparse
import logging

import torch
import torch.utils.data as data
import numpy as np

from utils.settings import Settings


settings_file_name = 'settings.ini'
settings = Settings(settings_file_name)
data_parameters = settings['DATA']
training_parameters = settings['TRAINING']
network_parameters = settings['NETWORK']
misc_parameters = settings['MISC']

number_of_modalities = 2
batch_size = 3
network_parameters['network_number'] = 1


# if data_parameters['t1t2ratio_flag'] == 2:
#     number_of_modalities += 1

if network_parameters['network_number'] == 1 or network_parameters['network_number']==2:
    data_parameters['fused_data_flag'] = True
else:
    data_parameters['fused_data_flag'] = False

    
if data_parameters['fused_data_flag'] == True:
    original_input_channels = number_of_modalities
    X = torch.rand(batch_size, number_of_modalities, 160, 192, 160)
else:
    original_input_channels = 1
    X = {}
    for idx in range(number_of_modalities):
        X[idx] = torch.rand(batch_size, 1, 160, 192, 160)

# if data_parameters['fused_data_flag'] == True or network_parameters['network_number']<=2:
#     original_input_channels = number_of_modalities
#     X = torch.rand(batch_size, number_of_modalities, 160, 192, 160)
# else:
#     original_input_channels = number_of_modalities
#     X = {}
#     for idx in range(number_of_modalities):
#         X[idx] = torch.rand(batch_size, 1, 160, 192, 160)

if network_parameters['network_number'] == 1:
    from MultiAgeMapper import AgeMapper_input1
    AgeMapperModel = AgeMapper_input1(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = original_input_channels,
                                network_parameters=network_parameters
                                )
    print('--> Using NETWORK 1')
elif network_parameters['network_number'] == 2:
    from MultiAgeMapper import AgeMapper_input2
    AgeMapperModel = AgeMapper_input2(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = original_input_channels,
                                network_2_modality_filter_outputs = network_parameters['network_2_modality_filter_outputs']
                                )
    print('--> Using NETWORK 2')
elif network_parameters['network_number'] == 3:
    from MultiAgeMapper import AgeMapper_input3
    AgeMapperModel = AgeMapper_input3(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities,
                                network_parameters=network_parameters
                                )
    print('--> Using NETWORK 3')
elif network_parameters['network_number'] == 4:
    from MultiAgeMapper import AgeMapper_input4
    AgeMapperModel = AgeMapper_input4(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities
                                )
    print('--> Using NETWORK 4')
elif network_parameters['network_number'] == 5:
    from MultiAgeMapper import AgeMapper_input5
    AgeMapperModel = AgeMapper_input5(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities,
                                network_parameters=network_parameters
                                )
    print('--> Using NETWORK 5')
elif network_parameters['network_number'] == 6:
    from MultiAgeMapper import AgeMapper_input6
    AgeMapperModel = AgeMapper_input6(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities
                                )
    print('--> Using NETWORK 6')
elif network_parameters['network_number'] == 7:
    from MultiAgeMapper import AgeMapper_input7
    AgeMapperModel = AgeMapper_input7(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities
                                )
    print('--> Using NETWORK 7')
elif network_parameters['network_number'] == 8:
    from MultiAgeMapper import AgeMapper_input8
    AgeMapperModel = AgeMapper_input8(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities
                                )
    print('--> Using NETWORK 8')
elif network_parameters['network_number'] == 9:
    from MultiAgeMapper import AgeMapper_input9
    AgeMapperModel = AgeMapper_input9(
                                fused_data_flag=data_parameters['fused_data_flag'],
                                original_input_channels = number_of_modalities
                                )
    print('--> Using NETWORK 9')
    
print(AgeMapperModel)

AgeMapperModel.eval()
with torch.no_grad():
    y = AgeMapperModel(X)
    print('y shape: ', y.shape)

# model_parameters = filter(lambda p: p.requires_grad, AgeMapperModel.parameters())
# params = sum([np.prod(p.size()) for p in model_parameters])
# print(params)